## load data

In [ ]:
# import dataframes
ca = pd.read_csv("s3://{}/raw/CA_youtube_trending_data.csv".format(bucket))

# import json
json_key = 'raw/CA_category_id.json'
response = s3.get_object(Bucket=bucket, Key=json_key)
json_data = response['Body'].read().decode('utf-8')

data = json.loads(json_data)
data_list = []
for item in data['items']:
    data_list.append({'id': item['id'], 'category': item['snippet']['title']})

df = pd.DataFrame(data_list)

## save into s3 bucket

In [ ]:
merged_df.to_csv('s3://{}/athena/files/final.csv'.format(bucket), index=False)

## Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

## Athena Table

In [ ]:
# staging directory
stagingdir = "s3://{}/athena/staging/".format(bucket)
conn = connect(region_name=region, s3_staging_dir=stagingdir)

# create a db
dbname = "viewboost"
sql = "CREATE DATABASE IF NOT EXISTS {}".format(dbname)
pd.read_sql(sql, conn)

# create table
tablename = 'table1'
s3_private_path_tsv = "s3://{}/athena/files/".format(bucket)

pd.read_sql("DROP TABLE IF EXISTS {}.{}".format(dbname,tablename), conn)
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
                location string,
                category string)
                
ROW FORMAT SERDE
  'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES ( 
  'escapeChar'='\"\"', 
  'quoteChar'='\"', 
  'separatorChar'=',') 
LOCATION '{}'
TBLPROPERTIES ("skip.header.line.count"="1");""".format(dbname, tablename, s3_private_path_tsv)

pd.read_sql(statement, conn)

### Example 1 - - Number of Videos per Category for Subset of Categories

In [ ]:
statement = """
SELECT category, COUNT(view_count) AS video_count
FROM {}.{}
GROUP BY category 
ORDER BY video_count DESC
""".format(
    database_name, table_name
)

df = pd.read_sql(statement, conn)
df.head()

### Example 2 -- Average View Count by Category

In [ ]:
import pandas as pd

statement = """
SELECT category, ROUND(AVG(view_count), -4) AS avg_view_count
FROM {}.{} 
GROUP BY category 
ORDER BY avg_view_count DESC
""".format(
    database_name, table_name
)

df = pd.read_sql(statement, conn)
df.head(5)

### Example 3 -- Trending video comment count over Time

In [ ]:
# SQL statement
statement = """
SELECT trending_month, trending_year, ROUND(AVG(comment_count),4)/ 1000 AS avg_comment
FROM {}.{}
GROUP BY trending_year, trending_month
ORDER BY trending_year, trending_month
""".format(
    database_name, table_name
)

df = pd.read_sql(statement, conn)
df['date'] = df['trending_month'] + '' + df['trending_year']
df.head(3)


# Visualize 

fig = plt.gcf()
fig.set_size_inches(12, 5)

fig.suptitle("Average Monthly Comment Count Over Time")

ax = plt.gca()
# ax = plt.gca().set_xticks(df['year'])
ax.locator_params(integer=True)
#ax.set_xticks(df["trending_month"])

df.plot(kind="line", x="date", y="avg_comment", color="red", ax=ax)

# plt.xticks(range(1995, 2016, 1))
# plt.yticks(range(0,6,1))
plt.xlabel("Date (MMYYYY)")
plt.ylabel("Average Comment Count (Thousands)")
plt.xticks(rotation=45)

# fig.savefig('average-rating.png', dpi=300)
plt.show()

### Athena Parquet Table

In [ ]:
# create parquet table
tablename_parquet = 'table1_parquet'
s3_private_path_parquet = "s3://{}/parquet".format(bucket)

pd.read_sql("DROP TABLE IF EXISTS {}.{}".format(dbname,tablename_parquet), conn)
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}', partitioned_by = ARRAY['category']) AS
SELECT location,
        category
FROM {}.{}""".format(
    dbname, tablename_parquet, s3_private_path_parquet, dbname, tablename
)
pd.read_sql(statement, conn)

# repair partitions
statement = "MSCK REPAIR TABLE {}.{}".format(dbname, tablename_parquet)
df = pd.read_sql(statement, conn)
df.head(5)

# shwo partitions
statement = "SHOW PARTITIONS {}.{}".format(dbname, tablename_parquet)
df_partitions = pd.read_sql(statement, conn)
df_partitions

## Seaborn setup

In [ ]:
sns.set(
    rc={
        "font.style": "normal",
        "axes.facecolor": "white",
        "grid.color": ".8",
        "grid.linestyle": "-",
        "figure.facecolor": "white",
        "figure.titlesize": 20,
        "text.color": "black",
        "xtick.color": "black",
        "ytick.color": "black",
        "axes.labelcolor": "black",
        "axes.grid": True,
        "axes.labelsize": 10,
        "xtick.labelsize": 10,
        "font.size": 10,
        "ytick.labelsize": 10,
    }
)


# from professor 
def show_values_barplot(axs, space):
    def _show_on_plot(ax):
        for p in ax.patches:
            _x = p.get_x() + p.get_width() + float(space)
            _y = p.get_y() + p.get_height()
            value = round(float(p.get_width()), 2)
            ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_plot(ax)
    else:
        _show_on_plot(axs)